In [25]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
from requests.exceptions import ConnectionError
# from pywebcopy import save_webpage

In [26]:

internal_urls = set()
external_urls = set()
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls

total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except:
        print("caught")


In [27]:
base_urls = ["http://www.longwood.edu/covid19/",
            "https://www.lynn.edu/coronavirus-faq",
            "https://www.madonna.edu/coronavirus/",
             "https://www.marian.edu/_temp/coronavirus",
            "https://www.marylhurst.edu/closure/",
            "http://www.marywood.edu/news/covid19faqs.html",
            "https://www.methodist.edu/coronavirus/",
            "https://mankato.mnsu.edu/about-the-university/news-and-events/covid-19-novel-coronavirus-and-the-campus-community/"]

In [28]:
for base_url in base_urls:
    internal_urls.add(base_url)
    parsedurl = urlparse(base_url)
    print(parsedurl)
    base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
    print(base_url_main)
    # base_url_text = base_url.split("//",1)[1]
    base_url_text = parsedurl.netloc+parsedurl.path
    print(base_url_text)
    # base_url_text_domain = base_url_text.split("/",1)[0]
    base_url_text_domain = parsedurl.netloc
    print(base_url_text_domain)
    crawl(base_url)
    print("[+] Total External links:", len(external_urls))
    print("[+] Total Internal links:", len(internal_urls))
    print("[+] Total:", len(external_urls) + len(internal_urls))
    count = 0
    last_count = 989874
    immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

    for i in range(0,4):
        for url in internal_urls:
            if base_url_text in url:
                immediate_urls.add(url)
        count = len(immediate_urls)
        print(count,i)
        if(count == last_count):
            break
        last_count = count
        for immediate_url in immediate_urls:
            try:
                crawl(immediate_url)
                print("[+] NEW Total External links:", len(external_urls))
                print("[+] NEW Total Internal links:", len(internal_urls))
                print("[+] NEW Total:", len(external_urls) + len(internal_urls))
                if len(internal_urls)>1000:
                    break
            except requests.exceptions.ConnectionError:
                continue

    def removeAfterN(yourStr, nth, occurenceOf):
        return occurenceOf.join(yourStr.split(occurenceOf)[:nth])
    
    
    semifinal_immediate_urls = set()

    for immediate_url in immediate_urls:
        stripped_url = removeAfterN(immediate_url,2,"http")


        semifinal_immediate_urls.add(stripped_url)
    #     final_immediate_urls.add(rest)
    #     print(rest)



    for i in semifinal_immediate_urls:
        print(i)
    total_count = len(semifinal_immediate_urls)
    print(total_count)
    
    
    final_immediate_urls = set()
    for semifinal_immediate_url in semifinal_immediate_urls:
        if "tel:" not in semifinal_immediate_url:
            final_immediate_urls.add(semifinal_immediate_url)


    for i in final_immediate_urls:
        print(i)
    total_count = len(final_immediate_urls)
    print(total_count)       
    
    
    
    parent_folder = "Masters" #Change based on folder
    name_of_folder = f'{base_url_text_domain}_Screenshots'
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
    if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
        os.makedirs(f'{parent_folder}\\{name_of_folder}')
    
    
    
    webpage_save = set()
    webpage_save_count = 0
    webpage_cannot_save = 0
    for i in final_immediate_urls:
        if ' ' in i:
            continue
        try:
            code=urlopen(i).getcode()
            webpage_save.add(i)
            webpage_save_count = webpage_save_count+1
            print(f"{code} can_save - {i}")
        except (urllib.error.HTTPError,urllib.error.URLError) as e:
            print("discarded")
            webpage_cannot_save = webpage_cannot_save+1
            continue

    print(f'Webpages save as HTML :{webpage_save_count}')
    print(f'Webpages unable to save as HTML :{webpage_cannot_save}')

    
    pdf_count = 0
    pdf_urls = set()
    for webpage in webpage_save:
        try:
            if webpage[-4:] == ".pdf":
                url = webpage
                file_name = url.replace(base_url,'')
                file_name = file_name.replace('/','')
                if file_name == '':
                    file_name = "Home"
                pdf_urls.add(webpage)
                print(webpage)
                r = requests.get(url, stream=True)
                with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
                    for chunk in r.iter_content(2000):
                        fd.write(chunk)
                pdf_count = pdf_count + 1
        except:
            print("caught")

    print(f"Pdfs saved: {pdf_count}")
    
    
    
    final_immediate_urls = final_immediate_urls - pdf_urls
    webpage_save = webpage_save - pdf_urls
    web_save_count = len(webpage_save)
    total_count = len(final_immediate_urls)
    print(total_count) 
    
    
    
    final_webpage_save_count = webpage_save_count - pdf_count
    print(final_webpage_save_count)
    
    
    webpage_save_counter_loop = 0
    for webpage in webpage_save:
        try:
            url = webpage
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                    file_name = "Home"
            urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
            webpage_save_counter_loop = webpage_save_counter_loop+1
            print(f'{webpage_save_counter_loop} of {final_webpage_save_count} - {webpage}')
        except:
            print("caught")

    print("Saving Completed")
    
    
    count = 0
    for i in final_immediate_urls:
        try:
            url = i
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                    file_name = "Home"
            count = count + 1
            print(f"{count} of {total_count}")
            print(f'Visiting {base_url_text}/{file_name}')
            print(f'...Taking a screenshot')
            driver.get(url)
            S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
            driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
            driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
            print(f'Screenshot of {file_name} page taken! \n')
        except:
            print("caught")
    #     print("Saving HTML")
    #     pyautogui.hotkey('ctrl', 's')
    #     time.sleep(1)
    #     pyautogui.typewrite(f'{file_name}.html')
    #     pyautogui.hotkey('enter')
    driver.quit()

    print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')

ParseResult(scheme='http', netloc='www.longwood.edu', path='/covid19/', params='', query='', fragment='')
http://www.longwood.edu
www.longwood.edu/covid19/
www.longwood.edu
[INT] Internal link: http://www.longwood.edu/
[INT] Internal link: http://www.longwood.edu/admissions/visit/
[INT] Internal link: http://www.longwood.edu/apply/
[INT] Internal link: http://www.longwood.edu/academics/
[INT] Internal link: http://www.longwood.edu/admissions
[INT] Internal link: http://www.longwood.edu/lancer-life/
[INT] Internal link: http://www.longwood.edu/about/
[INT] Internal link: http://www.longwood.edu/usersupport/
[INT] Internal link: http://www.longwood.edu/lancercard/
[INT] Internal link: http://www.longwood.edu/financialaid/
[INT] Internal link: http://www.longwood.edu/career/
[INT] Internal link: http://www.longwood.edu/recreation/
[INT] Internal link: http://www.longwood.edu/academicsuccess/
[INT] Internal link: http://www.longwood.edu/dec/
[INT] Internal link: http://www.longwood.edu/a-z

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: http://www.longwood.edu/health/student-information/services/
[INT] Internal link: http://www.longwood.edu/caps/
[INT] Internal link: http://www.longwood.edu/health/student-information/rights--responsibilities/
[INT] Internal link: http://www.longwood.edu/health/student-information/resources/
[INT] Internal link: http://www.longwood.edu/health/student-information/survival-kit/
[INT] Internal link: http://www.longwood.edu/caps/accreditation/
[INT] Internal link: http://www.longwood.edu/caps/services/group-counseling/
[INT] Internal link: http://www.longwood.edu/caps/services/adhd--learning-disability-testing/
[INT] Internal link: http://www.longwood.edu/caps/services/outreach/
[INT] Internal link: http://www.longwood.edu/caps/services/seasonal-affective-disorder-sad/
[INT] Internal link: http://www.longwood.edu/caps/services/
[INT] Internal link: http://www.longwood.edu/caps/faq/
[INT] Internal link: http://www.longwood.edu/caps/caps-team/
[INT] Internal link: http:/

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: http://www.longwood.edu/disability/
[INT] Internal link: http://www.longwood.edu/housing/
[INT] Internal link: http://www.longwood.edu/disability/accommodations/
[INT] Internal link: http://www.longwood.edu/media/disability-resources/public-site/New_Registration_Form-1.pdf
[INT] Internal link: http://www.longwood.edu/media/disability-resources/public-site/Verification_of_Disability_Form.pdf
[INT] Internal link: http://www.longwood.edu/media/disability-resources/public-site/Longwood-University-ADD-FORM.pdf
[INT] Internal link: http://www.longwood.edu/media/disability-resources/public-site/Accommodations-Request-Form.pdf
[INT] Internal link: http://www.longwood.edu/media/disability-resources/public-site/Alternate-Format-Request.pdf
[INT] Internal link: http://www.longwood.edu/disability/about/student-test-request-form/
[INT] Internal link: http://www.longwood.edu/media/disability-resources/public-site/Student-Guide---ODR-Testing-Center.pdf
[INT] Internal link: http:/

...

[+] Total External links: 391
[+] Total Internal links: 409
[+] Total: 800
1 0
[+] NEW Total External links: 391
[+] NEW Total Internal links: 409
[+] NEW Total: 800
1 1
https://www.marylhurst.edu/closure/
1
https://www.marylhurst.edu/closure/
1
discarded
Webpages save as HTML :0
Webpages unable to save as HTML :1
Pdfs saved: 0
1
0
Saving Completed
1 of 1
Visiting www.marylhurst.edu/closure//Home
...Taking a screenshot
Screenshot of Home page taken! 

Task Completed! Files stored in the Masters\www.marylhurst.edu_Screenshots Folder
ParseResult(scheme='http', netloc='www.marywood.edu', path='/news/covid19faqs.html', params='', query='', fragment='')
http://www.marywood.edu
www.marywood.edu/news/covid19faqs.html
www.marywood.edu
[INT] Internal link: http://www.marywood.edu
[INT] Internal link: http://www.marywood.edu/admissions/applying
[INT] Internal link: http://www.marywood.edu/admissions/visit-campus/index.html
[INT] Internal link: http://www.marywood.edu/request
[INT] Internal link:

[+] NEW Total External links: 454
[+] NEW Total Internal links: 638
[+] NEW Total: 1092
[INT] Internal link: https://www.methodist.edu/category/uncategorized/
[INT] Internal link: https://www.methodist.edu/coronavirus/faq
[INT] Internal link: https://www.methodist.edu/coronavirus/faq/tel:19106307151
[INT] Internal link: https://www.methodist.edu/coronavirus/faq/tel:19106307264
[INT] Internal link: https://www.methodist.edu/coronavirus/faq/tel:19106307300
[INT] Internal link: https://www.methodist.edu/coronavirus/faq/tel:19106307123
[INT] Internal link: https://www.methodist.edu/coronavirus/faq/tel:18444888395
[INT] Internal link: https://www.methodist.edu/coronavirus/faq/tel:18558468376
[INT] Internal link: https://www.methodist.edu/coronavirus/faq/tel:18557656995
[INT] Internal link: https://www.methodist.edu/coronavirus/faq/tel:19106307515
[INT] Internal link: https://www.methodist.edu/wp-content/uploads/2020/03/caring.pdf
[INT] Internal link: https://www.methodist.edu/february-20-20

[INT] Internal link: https://mankato.mnsu.edu/
[INT] Internal link: https://mankato.mnsu.edu/future-students/undergraduate/visit-campus/
[INT] Internal link: https://mankato.mnsu.edu/future-students/apply/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/news-and-events/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/maps-and-transportation/directories/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/find-us/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/maps-and-transportation/contacts/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/fast-facts/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/maps-and-transportation/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/office-of-the-president/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/diversity-and-inclusion/
[INT] 

[INT] Internal link: https://mankato.mnsu.edu/about-the-university/news-and-events/covid-19-novel-coronavirus-and-the-campus-community/return-to-learn/timeline-and-planning/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/news-and-events/covid-19-novel-coronavirus-and-the-campus-community/return-to-learn/flexsync-classrooms/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/news-and-events/covid-19-novel-coronavirus-and-the-campus-community/return-to-learn/timeline-and-planning/action-team-structure-and-leaders/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/news-and-events/covid-19-novel-coronavirus-and-the-campus-community/return-to-learn/safety-measures/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/news-and-events/covid-19-novel-coronavirus-and-the-campus-community/return-to-learn/return-to-learn-announcements/
[INT] Internal link: https://mankato.mnsu.edu/about-the-university/news-and-events/covid-19-nove

discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
Webpages save as HTML :0
Webpages unable to save as HTML :23
Pdfs saved: 0
23
0
Saving Completed
1 of 23
Visiting mankato.mnsu.edu/about-the-university/news-and-events/covid-19-novel-coronavirus-and-the-campus-community//covid-19-announcements
...Taking a screenshot
Screenshot of covid-19-announcements page taken! 

2 of 23
Visiting mankato.mnsu.edu/about-the-university/news-and-events/covid-19-novel-coronavirus-and-the-campus-community//coronavirus-frequently-asked-questionsgeneral-coronavirus-questions
...Taking a screenshot
Screenshot of coronavirus-frequently-asked-questionsgeneral-coronavirus-questions page taken! 

3 of 23
Visiting mankato.mnsu.edu/about-the-university/news-and-events/covid-19-novel-coronavirus-and-the-campus-community//return-to-learn
.

In [29]:
print("DONE")

DONE
